<a href="https://colab.research.google.com/github/mkbahk/QuantumComputing/blob/main/QuantumSubroutineAlgorithms_QAE_Claude_mkbahk_20250812.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install qiskit[visualization]==1.2.4
%pip install qiskit-aer==0.15.1
%pip install git+https://github.com/qiskit-community/qiskit-textbook.git#subdirectory=qiskit-textbook-src

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 7.7 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=8ddc9bed50a2816b993c55191cf97bd124b3d338a1891bd8f6513a9dd582bb5c
  Stored in directory: /root/.cache/pip/wheels/b1/7a/33/9fdd892f784ed4afda62b685ae3703adf4c91aa0f524c28f03
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 26.4 MB/s eta 0:00:00
  Cloning https://github.com/qiskit-community/qiskit-textbook.git to /tmp/pip-req-build-xq07ny_t
  Running command

In [4]:
%pip install qiskit-algorithms

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 7.5 MB/s eta 0:00:00


In [5]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile, assemble
from qiskit.quantum_info import Statevector, Operator
from qiskit.visualization import plot_histogram, plot_state_city, plot_bloch_multivector
from qiskit_textbook.tools import array_to_latex
from qiskit_aer import Aer, AerSimulator
from IPython.display import display, Math
import numpy as np

#from qiskit.primitives import Sampler

In [17]:
from math import asin, sqrt

#from qiskit import QuantumCircuit
from qiskit.primitives import Sampler
from qiskit_algorithms import AmplitudeEstimation, EstimationProblem
from qiskit_algorithms.amplitude_estimators import MaximumLikelihoodAmplitudeEstimation

In [18]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
from qiskit.circuit.library import GroverOperator
from qiskit.quantum_info import Statevector
import matplotlib.pyplot as plt
from typing import Optional, Union

In [32]:
class QuantumAmplitudeEstimation:
    """
    Quantum Amplitude Estimation (QAE) implementation using Qiskit

    QAE estimates the amplitude a of a quantum state |ψ⟩ = sqrt(1-a²)|ψ₀⟩ + sqrt(a²)|ψ₁⟩
    where |ψ₁⟩ represents the "good" states we want to measure.
    """

    def __init__(self, state_preparation: QuantumCircuit, good_state: Union[str, list]):
        """
        Initialize QAE

        Args:
            state_preparation: Quantum circuit that prepares the initial state
            good_state: Binary string or list defining which states are "good"
        """
        self.state_preparation = state_preparation
        self.good_state = good_state
        self.num_qubits = state_preparation.num_qubits
    ###def

    def create_oracle(self) -> QuantumCircuit:
        """
        Create oracle that flips the phase of good states
        """
        oracle = QuantumCircuit(self.num_qubits, name='Oracle')
        z_gate = QuantumCircuit(1).z(0).to_instruction()

        if isinstance(self.good_state, str):
            # Single good state case
            for i, bit in enumerate(self.good_state):
                if bit == '0':
                    oracle.x(i)
                ###if
            ###for
            # Apply multi-controlled Z gate
            controlled_z = z_gate.control(self.num_qubits - 1)
            oracle.append(controlled_z, range(self.num_qubits))

            for i, bit in enumerate(self.good_state):
                if bit == '0':
                    oracle.x(i)
                ###if
            ###for
        else:
            # Multiple good states case
            for state in self.good_state:
                temp_oracle = QuantumCircuit(self.num_qubits)
                for i, bit in enumerate(state):
                    if bit == '0':
                        temp_oracle.x(i)
                    ###if
                ###for
                # Apply multi-controlled Z gate
                controlled_z = z_gate.control(self.num_qubits - 1)
                temp_oracle.append(controlled_z, range(self.num_qubits))

                for i, bit in enumerate(state):
                    if bit == '0':
                        temp_oracle.x(i)
                    ###if
                ###for
                oracle.compose(temp_oracle, inplace=True)
            ###for
        ###if

        return oracle
    ###def


    def create_diffuser(self) -> QuantumCircuit:
        """
        Create diffuser (inversion about average) operator
        """
        diffuser = QuantumCircuit(self.num_qubits, name='Diffuser')
        z_gate = QuantumCircuit(1).z(0).to_instruction()

        # Apply inverse of state preparation
        diffuser.append(self.state_preparation.inverse(), range(self.num_qubits))

        # Flip phase of |0...0⟩ state
        diffuser.x(range(self.num_qubits))
        # Apply multi-controlled Z gate
        controlled_z = z_gate.control(self.num_qubits - 1)
        diffuser.append(controlled_z, range(self.num_qubits))
        diffuser.x(range(self.num_qubits))

        # Apply state preparation again
        diffuser.append(self.state_preparation, range(self.num_qubits))

        return diffuser
    ###def

    def create_grover_operator(self) -> QuantumCircuit:
        """
        Create Grover operator Q = -A*S₀*A†*Sχ
        where A is state preparation, S₀ is diffuser, Sχ is oracle
        """
        oracle = self.create_oracle()
        diffuser = self.create_diffuser()

        grover_op = QuantumCircuit(self.num_qubits, name='Q')
        grover_op.append(oracle, range(self.num_qubits))
        grover_op.append(diffuser, range(self.num_qubits))

        return grover_op
    ###def

    def run_qae(self, m_values: list, shots: int = 8192) -> dict:
        """
        Run QAE algorithm with different numbers of Grover iterations

        Args:
            m_values: List of numbers of Grover iterations to try
            shots: Number of measurement shots

        Returns:
            Dictionary with results for each m value
        """
        grover_op = self.create_grover_operator()
        simulator = AerSimulator()
        results = {}

        for m in m_values:
            # Create QAE circuit
            qc = QuantumCircuit(self.num_qubits, self.num_qubits)

            # Apply state preparation
            qc.append(self.state_preparation, range(self.num_qubits))

            # Apply Grover operator m times
            for _ in range(m):
                qc.append(grover_op, range(self.num_qubits))
            ###for

            # Measure all qubits
            qc.measure_all()

            # Run simulation
            transpiled_qc = transpile(qc, simulator)
            job = simulator.run(transpiled_qc, shots=shots)
            counts = job.result().get_counts()

            # Calculate probability of measuring good states
            good_counts = 0
            total_counts = sum(counts.values())

            for state, count in counts.items():
                if isinstance(self.good_state, str):
                    if state == self.good_state:
                        good_counts += count
                    ###if
                else:
                    if state in self.good_state:
                        good_counts += count
                    ###if
                ###if
            ###for

            prob_good = good_counts / total_counts
            results[m] = {
                'probability': prob_good,
                'counts': counts,
                'estimated_amplitude': np.sqrt(prob_good)
            }
        #for

        return results
    ###def

    def estimate_amplitude(self, max_iterations: int = 10) -> float:
        """
        Estimate amplitude using phase estimation approach

        Args:
            max_iterations: Maximum number of iterations for phase estimation

        Returns:
            Estimated amplitude
        """
        # This is a simplified version - full QAE would use quantum phase estimation
        m_values = list(range(1, max_iterations + 1))
        results = self.run_qae(m_values, shots=8192)

        # Find the best estimate based on the pattern
        best_m = 1
        best_score = float('inf')

        for m, result in results.items():
            prob = result['probability']
            # Look for oscillating pattern characteristic of QAE
            expected_prob = (np.sin((2*m+1)*np.arcsin(0.5)))**2  # Assuming a=0.5 for demo
            score = abs(prob - expected_prob)
            if score < best_score:
                best_score = score
                best_m = m
            ###if
        ###for

        return results[best_m]['estimated_amplitude']
    ###def
###class

def create_example_state_preparation(num_qubits: int) -> QuantumCircuit:
    """
    Create an example state preparation circuit
    This creates a superposition where some states have higher amplitude
    """
    qc = QuantumCircuit(num_qubits)

    # Apply Hadamard to create superposition
    qc.h(range(num_qubits))

    # Apply some rotations to create non-uniform amplitudes
    for i in range(num_qubits):
        qc.ry(np.pi/4 * (i+1), i)

    return qc
###def

In [29]:
def main():
    """
    Demonstrate QAE with a simple example
    """
    print("=== Quantum Amplitude Estimation Demo ===\n")

    # Setup
    num_qubits = 3
    state_prep = create_example_state_preparation(num_qubits)

    # Define good states (states we want to measure the amplitude for)
    good_states = ['001', '010', '100']  # Example good states

    # Create QAE instance
    qae = QuantumAmplitudeEstimation(state_prep, good_states)

    # Calculate theoretical amplitude for comparison
    # Create the actual quantum state to get the true amplitude
    qc_theory = QuantumCircuit(num_qubits)
    qc_theory.append(state_prep, range(num_qubits))

    statevector = Statevector.from_instruction(qc_theory)
    probs = statevector.probabilities()

    true_prob_good = 0
    for state in good_states:
        state_index = int(state, 2)
        true_prob_good += probs[state_index]
    ###for

    true_amplitude = np.sqrt(true_prob_good)
    print(f"True amplitude (theoretical): {true_amplitude:.4f}")
    print(f"True probability of good states: {true_prob_good:.4f}\n")

    # Run QAE with different numbers of iterations
    m_values = [1, 2, 3, 4, 5]
    results = qae.run_qae(m_values, shots=8192)

    print("QAE Results:")
    print("Iterations | Measured Prob | Estimated Amplitude | Error")
    print("-" * 60)

    amplitudes = []
    errors = []

    for m, result in results.items():
        prob = result['probability']
        est_amp = result['estimated_amplitude']
        error = abs(est_amp - true_amplitude)

        amplitudes.append(est_amp)
        errors.append(error)

        print(f"{m:^10} | {prob:^13.4f} | {est_amp:^19.4f} | {error:.4f}")
    ###for

    # Find best estimate
    best_idx = np.argmin(errors)
    best_m = m_values[best_idx]
    best_amplitude = amplitudes[best_idx]

    print(f"\nBest estimate: {best_amplitude:.4f} (using {best_m} iterations)")
    print(f"Error: {errors[best_idx]:.4f}")

    # Plot results
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(m_values, amplitudes, 'bo-', label='Estimated Amplitude')
    plt.axhline(y=true_amplitude, color='r', linestyle='--', label='True Amplitude')
    plt.xlabel('Number of Grover Iterations (m)')
    plt.ylabel('Estimated Amplitude')
    plt.title('QAE Amplitude Estimates')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(1, 2, 2)
    plt.plot(m_values, errors, 'ro-')
    plt.xlabel('Number of Grover Iterations (m)')
    plt.ylabel('Estimation Error')
    plt.title('QAE Estimation Error')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print("\n=== Example: Measuring specific measurement outcomes ===")
    for m in [1, 3, 5]:
        result = results[m]
        print(f"\nWith {m} Grover iterations:")
        sorted_counts = sorted(result['counts'].items(), key=lambda x: x[1], reverse=True)
        for state, count in sorted_counts[:5]:  # Show top 5 results
            prob = count / sum(result['counts'].values())
            marker = " ← Good state" if state in good_states else ""
            print(f"  |{state}⟩: {count:4d} counts ({prob:.3f}){marker}")
        ###for
    ###for
###def

In [33]:
if __name__ == "__main__":
    main()
###if

=== Quantum Amplitude Estimation Demo ===

True amplitude (theoretical): 0.1464
True probability of good states: 0.0214



AttributeError: 'InstructionSet' object has no attribute 'to_instruction'